# FER with CNN on optical flows

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import h5py
import cv2
from sklearn.metrics import zero_one_loss

#Pytorch learning
import torch
from torchsummary import summary
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
import torch.optim as optim
import torch.nn.functional as F

from exp_rec import CNN as ExpNet

#Functions import
from functions import get_imflow
from functions import write_results

## Data loading

In [13]:
#######################################################################
# Loading data tables from H5py files
#######################################################################

h5f = h5py.File('train/data_train_images.h5','r')
train_images = h5f['data_train_images'][:]
h5f.close()

h5f = h5py.File('test/data_test_images.h5','r')
test_images = h5f['data_test_images'][:]
h5f.close()

h5f = h5py.File('train/data_train_labels.h5','r')
train_labels = h5f['data_train_labels'][:]
train_labels = train_labels
h5f.close()

In [14]:
train_images.shape

(540, 10, 300, 200)

In [16]:
trainset = {}
train_labels = torch.Tensor(train_labels).long()
for i in range(len(train_labels)):
    trainset[i] = (get_imflow(train_images[i][0],train_images[i][9]).transpose(2,0,1),train_labels[i])

In [17]:
testset = []
for i in range(len(test_images)):
    testset.append(get_imflow(test_images[i][0],test_images[i][9]).transpose(2,0,1))

In [18]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

In [14]:
train_flows = []
for i in range(len(train_labels)):
    train_flows.append(get_imflow(train_images[i][0],train_images[i][9]).transpose(2,0,1))

## Model architecture

In [15]:
batch_size = 16

In [19]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Upsampling
        self.down = nn.Sequential(
            nn.Conv2d(2, 8, 3, padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2)
            )
        # Upsampling
        self.up = nn.Sequential(
            nn.Linear(29600,256),
            nn.ReLU(True),
            nn.Linear(256,6),
            nn.Softmax(dim=1)
            )
        

    def forward(self, img):
        out = self.down(img)
        out = self.up(out.view(out.size(0), -1))
        return out

In [52]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)

## Training

In [51]:
cnn = CNN()

In [53]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data[0]
        labels = data[1]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 4:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,     5] loss: 0.004
[1,    10] loss: 0.004
[1,    15] loss: 0.004
[1,    20] loss: 0.004
[1,    25] loss: 0.004
[1,    30] loss: 0.004
[2,     5] loss: 0.003
[2,    10] loss: 0.003
[2,    15] loss: 0.003
[2,    20] loss: 0.003
[2,    25] loss: 0.003
[2,    30] loss: 0.003
[3,     5] loss: 0.003
[3,    10] loss: 0.003
[3,    15] loss: 0.003
[3,    20] loss: 0.003
[3,    25] loss: 0.003
[3,    30] loss: 0.003
[4,     5] loss: 0.003
[4,    10] loss: 0.003
[4,    15] loss: 0.003
[4,    20] loss: 0.003
[4,    25] loss: 0.003
[4,    30] loss: 0.003
[5,     5] loss: 0.003
[5,    10] loss: 0.003
[5,    15] loss: 0.003
[5,    20] loss: 0.003
[5,    25] loss: 0.003
[5,    30] loss: 0.003
[6,     5] loss: 0.003
[6,    10] loss: 0.003
[6,    15] loss: 0.003
[6,    20] loss: 0.003
[6,    25] loss: 0.003
[6,    30] loss: 0.003
[7,     5] loss: 0.003
[7,    10] loss: 0.003
[7,    15] loss: 0.003
[7,    20] loss: 0.003
[7,    25] loss: 0.003
[7,    30] loss: 0.003
[8,     5] loss: 0.003
[8,    10] 

In [13]:
torch.save(cnn.state_dict(), "cnn_1.pth")

## Test model

In [21]:
cnn = CNN()
cnn.load_state_dict(torch.load("cnn_1.pth"))

<All keys matched successfully>

In [24]:
test_images = torch.Tensor(testset)

In [25]:
outputs = cnn(test_images)
_, predicted = torch.max(outputs, 1)

In [27]:
len(predicted)

131

In [29]:
write_results(predicted,"results_2")